In [1]:
import os
from pathlib import Path

# Verify the new working directory
print("Current working directory:", os.getcwd())

# Get the current notebook directory
current_dir = Path().resolve()

# Change to a relative path, e.g., go to a 'data' subdirectory relative to the current notebook
relative_path = current_dir / '../../code/'

# Change the working directory
os.chdir(relative_path)

# Verify the new working directory
print("Current working directory:", os.getcwd())

Current working directory: /cephfs/farzan/runsaffu
Current working directory: /cephfs/code


In [2]:
exec(open("saffu/configuration_saffu.py").read())
exec(open("saffu/tokenization_saffu.py").read())
exec(open("saffu/utilities_saffu.py").read())
exec(open("saffu/data_saffu.py").read())
exec(open("saffu/modeling_saffu.py").read())
exec(open("saffu/training_saffu.py").read())
exec(open("saffu/inference_saffu.py").read())
exec(open("saffu/tuning_saffu.py").read())
exec(open("saffu/load_data.py").read())

logger = logging.get_logger(__name__)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
devstr = "cuda:0" # "cpu" #

gpu = False if (devstr == 'cpu') else True
device = 'cpu' if (devstr == 'cpu') else (torch.device(devstr if torch.cuda.is_available() else 'cpu') 
                                          if devstr else torch.cuda.current_device())

In [29]:
# determine the data set(s)
data_set = "helpful-base" # +harmless-base+babylm_10M+babylm_100M+BWB
model_size = "tiny" # "tiny" # "micro" # "small" # "big" #
training_sizes = {"helpful-base": 5, "harmless-base": 5, "babylm_10M": 10, "babylm_100M": 100, "BWB": 1000} # in millions of word-tokens
devsample = 10 # dev data is 1/10 of total available
dataset_size = sum([training_sizes[data_subset] for data_subset in data_set.split("+")])
downsample = int(dataset_size/5) # roughly 5 million word-tokens per split
eta = 0.05 # 0.05
docs, ddocs, tdocs = [], [], []
convos, dconvos, tconvos = [], [], []

In [30]:
if ("helpful-base" in data_set) or ("harmless-base" in data_set):
    train_conversations = []; dev_conversations = []; test_conversations = []
    # Load one of the helpful subsets
    if "helpful-base" in data_set:
        train_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/helpful_train.json")
        dev_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/helpful_dev.json")
        # test_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/helpful_test.json")
    # Load one of the harmless subsets
    if "harmless-base" in data_set:
        train_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/harmless_train.json")
        dev_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/harmless_dev.json")
        # test_conversations += load_hh_rlhf("/cephfs/data/hh_rlhf/backup/harmless_test.json")
    
    convos, dconvos, test_threads = defaultdict(list), defaultdict(list), defaultdict(list)
    for conversation in train_conversations:
        initial_thread = "".join([turn[0] + turn[1] for turn in conversation['chosen'][:2]])
        convos[initial_thread].append((len(conversation['chosen']), conversation['chosen']))
    for conversation in dev_conversations:
        initial_thread = "".join([turn[0] + turn[1] for turn in conversation['chosen'][:2]])
        dconvos[initial_thread].append((len(conversation['chosen']), conversation['chosen']))
    
    convos = [sorted(convos[initial_thread], reverse = True)[0][1] for initial_thread in convos]
    dconvos = [sorted(dconvos[initial_thread], reverse = True)[0][1] for initial_thread in dconvos]
    
    docs = []
    for conversation in train_conversations:
        docs += [x[1] for x in conversation['chosen'][-2:]] + [conversation['rejected'][-1][1]]
    
    print("Numbers of training and development threads: ", len(convos), len(dconvos))

Numbers of training and development threads:  27281 2280


In [31]:
if ("babylm_10M" in data_set) or ("babylm_100M" in data_set):
    if ("babylm_10M" in data_set):
        for fname in tqdm(os.listdir("/cephfs/data/babylm_data/babylm_10M/"), desc = "Loading 10M training tokens"):
            if ".train" == fname[-6:]:
                docs, convos = load_BBLM("/cephfs/data/babylm_data/babylm_10M/", fname, docs, convos)
    if ("babylm_100M" in data_set):
        for fname in tqdm(os.listdir("/cephfs/data/babylm_data/babylm_100M/"), desc = "Loading 100M training tokens"):
            if ".train" == fname[-6:]:
                docs, convos = load_BBLM("/cephfs/data/babylm_data/babylm_100M/", fname, docs, convos)
    for fname in tqdm(os.listdir("/cephfs/data/babylm_data/babylm_dev/"), desc = "Loading development tokens"):
        if ".dev" == fname[-4:]:
            ddocs, dconvos = load_BBLM("/cephfs/data/babylm_data/babylm_dev/", fname, ddocs, dconvos)
    # for fname in tqdm(os.listdir("/cephfs/data/babylm_data/babylm_test/"), desc = "Loading testing tokens"):
    #     if ".test" == fname[-5:]:
    #         tdocs, tdocs = load_BBLM("/cephfs/data/babylm_data/babylm_test/", fname, tdocs, tconvos)

In [32]:
if "BWB" in data_set:
    bwb_train_docs, bwb_test_docs = load_bwb("/cephfs/data/bwb/datasets/")
    docs += bwb_train_docs
    # dconvos += [[["Human: ", x + "\n\n"]] for x in tqdm(bwb_test_docs, desc = "Loading BWB testing texts") if x]
    convos += [[["Human: ", x + "\n\n"]] for x in tqdm(bwb_train_docs, desc = "Loading BWB training texts") if x]
    del bwb_train_docs, bwb_test_docs

In [33]:
config = get_config(model_size = model_size)
tokenizer_name = f"{data_set}-{model_size}"
tokenizer = SAFFUTokenizer(config)
tokenizer_directory = "./cache/"
vocab_file = os.path.join(tokenizer_directory, tokenizer._model_path,
                          (tokenizer_name + "-" if tokenizer_name else "") + "vocab.json")
reload = False
if not reload and os.path.exists(vocab_file):
    print(f"Loading tokenizer: {tokenizer_name}")
    result = tokenizer.load(tokenizer_name, load_directory = tokenizer_directory)
else:
    print(f"Training tokenizer: {tokenizer_name}")
    tokenizer.train(tokenizer.pretokenize_documents(docs))
    tokenizer.save_vocabulary(tokenizer_name, save_directory = tokenizer_directory)

tokenizer.set_vocabulary()
print("Vocabulary size for experiment: ", len(tokenizer._vocabulary))

if ("Assistant: " not in tokenizer._vocabulary) and ("Human: " not in tokenizer._vocabulary):
    tokenizer.augment_vocabulary(["Assistant: ", "Human: "])
data_file = os.path.join(tokenizer_directory, tokenizer._model_path,
                         (tokenizer_name + "-" if tokenizer_name else "") + 
                         f"data-space_{tokenizer.config._space}-r_{tokenizer.config._r}-b_{tokenizer.config._b}-heads_{tokenizer.config._heads}-N_{tokenizer.config._N}.json")

tokenizer._tokenize("These casseroles disgust Kayla."), [x in tokenizer._vocabulary for x in tokenizer._tokenize("These casseroles disgust Kayla.")]

Loading tokenizer: helpful-base-tiny
Portion of model's 87646 reference tokens covered: 0.9995664377153549
Vocabulary size for experiment:  6387


(['The',
  'se',
  ' ca',
  'ss',
  'er',
  'ol',
  'es',
  ' dis',
  'gu',
  'st',
  ' K',
  'ay',
  'la',
  '.'],
 [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True])

In [34]:
convos

[[['Human: ', 'Hi, I want to learn to play horseshoes. Can you teach me?\n\n'],
  ['Assistant: ',
   'I can, but maybe I should begin by telling you that a typical game consists of 2 players and 6 or 8 horseshoes.\n\n'],
  ['Human: ',
   'Okay. What else is needed to play, and what are the rules?\n\n'],
  ['Assistant: ',
   'A horseshoe is usually made out of metal and is about 3 to 3.5 inches long and around 1 inch thick. The horseshoe should also have a 2 inch by 3 inch flat at the bottom where the rubber meets the metal. We also need two stakes and six horseshoes.\n\n'],
  ['Human: ', 'Okay, horseshoes, and two stakes. How do we play?\n\n'],
  ['Assistant: ',
   'Well, each player will use a hand to throw a horseshoe, and then when it hits the ground, a line is drawn where it lands. The horseshoe that falls closest to the stake is a “winner.” If no one gets the closest horseshoe, then that round is a “tie” and players go again until there is a “winner”.']],
 [['Human: ', 'How do I t

In [28]:
dconvos

[]

```
Epoch-0 perplexity [900.360501479926, 2178.3533993188635, 79212.25031167455, 99761.9505256068] (dim 1, even sectors)
Epoch-0 perplexity [496.76525781573207, 533.2700570397158, 100389.77601949903, 100392.99681093436] (dim 1, even sectors, all data)
Epoch-0 perplexity [913.0182298019313, 2291.6172177211765, 81902.45084486288, 99930.46726348072] (dim 0, odd sectors, transpose)
Epoch-0 perplexity [836.1772613983708, 1946.7510791982704, 81318.80639670715, 99894.95645510106] (dim 0, odd sectors)
Epoch-0 perplexity [987.3555092003801, 2279.969741016503, 81942.77913312569, 99931.26067729699] (dim 1, odd sectors, no icf)
Epoch-0 perplexity [763.220903806218, 1855.6688927998707, 81153.24557677505, 99884.87657541053] (dim 1, odd sectors)
Epoch-0 perplexity [592.3969722894352, 628.1477425335485, 100404.3056072563, 100409.60345968607] (dim 1, odd sectors, all data)
```

In [7]:
exec(open("saffu/modeling_saffu.py").read())
model = SAFFUDecoder(config, SAFFUEncoder(config, tokenizer)).to(device)
stage = "init"; reload = False
if reload or (not os.path.exists(f"./models_to_test/{data_set}-{model_size}-{stage}.state")):
    save_model(model, data_set, model_size, stage)

In [8]:
exec(open("saffu/tuning_saffu.py").read())
total_params, total_learnable = 0, 0
for name, param in model.named_parameters():
    total_params += int(np.exp(sum(np.log(param.shape))))
    if param.requires_grad:
        total_learnable += int(np.exp(sum(np.log(param.shape)))) # param.shape[0]*param.shape[1]
        # print(name, param.shape[0]*param.shape[1])

print(f"Total numbers of learnable/all parameters: {total_learnable}/{total_params}")

model.encoder._V.weight.requires_grad = False

seed = 691; ignore_space = False; ignore_case = False; warm_start = True; verbose = True; 
warm_vecs = 1*(2**0 + 0.99999); identity_ratio = 2**(-1); icf = True 
log_label = False; nlabels = 1*(2**0); centroids = False; label_iterations = 1*(2**10) # None 
epochs = int(np.max([int(downsample/5), 1]))*(2**5)
patience = 2**1
reload = False
warm_file = "".join([data_file[:-5] + "-", 
                     f"b_{tokenizer.config._bits}-hb_{tokenizer.config._hidden}-",
                     f"we_{int(tokenizer.config._wave_encode)}-oa_{tokenizer.config._o_agg}-ra_{tokenizer.config._r_agg}-ba_{tokenizer.config._b_agg}-",
                     f"mr_{int(tokenizer.config._mask_r)}-mb_{int(tokenizer.config._mask_b)}-md_{tokenizer.config._model_documents}-",
                     f"is_{int(ignore_space)}-ic_{int(ignore_case)}-ws_{int(warm_start)}-wv_{int(warm_vecs)}-ds_{downsample}-seed_{seed}"])
tuner = SAFFUTuner(ignore_case, ignore_space, devstr, warm_vecs, identity_ratio = identity_ratio, 
                   label_iterations = label_iterations, log_label = log_label, nlabels = nlabels, centroids = centroids, icf = icf)
tuner.warm_start(model, convos, dconvos, downsample*10, seed, epochs, patience, devsample = devsample, model_file = warm_file, reload = reload, verbose = verbose)

Total numbers of learnable/all parameters: 44722873/45252277
Training pads per block, pads to tokens, portion padding:  89.608 0.09589979366261697 0.0875078125
Testing pads per block, pads to tokens, portion padding:  83.00877192982456 0.08821418250631566 0.0810632538377193
Epoch-0 perplexity [1206.8975663905253, 1543.9671058747447, 3862.2552129191236, 580070.218273425] <sod>Human: What you., do
 what are, be



Assistant: I you're the good, for an
 do,
Epoch-1 perplexity [859.108905690172, 930.7398331187399, 175347.74474764962, 1155295.0044960012] <sod>Human: How you can really in
 also can,s, for a,.

Assistant: Yeah,. s thes,
Epoch-2 perplexity [698.670368572536, 731.3916744345651, 1066035.0111860833, 1542281.277128267] <sod>Human: How you can been of
 an can,s, for a,.

Assistant:  , in what the about,
Epoch-3 perplexity [634.3206189406983, 656.9123920521052, 2908308.600961864, 1498811.6613692269] <sod>Human: The you’ the I the by can, about a different, the, even a what, or “, som

In [9]:
stage = "start"; reload = True
if reload or (not os.path.exists(f"./models_to_test/{data_set}-{model_size}-{stage}.state")):
    save_model(model, data_set, model_size, stage)
    with open(f"./models_to_test/{data_set}-{model_size}-init-docsizes.json", "w") as f:
        f.write(json.dumps(model.docsizes))
    with open(f"./models_to_test/{data_set}-{model_size}-init-reference.json", "w") as f:
        f.write(json.dumps(model.reference))

In [10]:
tune_loss = model._losses; del tuner
tune_loss

[[25252.000000000047, 25252.0, 101008.99999999991, 101009.00000000028],
 [1206.8975663905253,
  1543.9671058747447,
  3862.2552129191236,
  580070.218273425],
 [859.108905690172, 930.7398331187399, 175347.74474764962, 1155295.0044960012],
 [698.670368572536, 731.3916744345651, 1066035.0111860833, 1542281.277128267],
 [634.3206189406983, 656.9123920521052, 2908308.600961864, 1498811.6613692269],
 [596.409158098091, 612.5116187196204, 5617404.368630349, 804332.062000717],
 [565.5916733421568, 581.1854545939651, 9037018.091541408, 555731.5409881896],
 [562.7869340592322, 567.9691474673151, 13010483.347625697, 282827.4080418706],
 [545.0691530255492, 551.1026175665359, 17431722.817370802, 180010.7134817856],
 [534.3093461702189,
  546.2979465046792,
  22192768.187902234,
  110849.67145533318],
 [527.6059473332663, 540.54562969348, 27233161.264319476, 60084.45593499909],
 [499.02590424777406,
  536.6407168435882,
  13676.111069708226,
  53636.54586908587],
 [501.1792260444718,
  533.8062175

In [11]:
def noise_matrix(matrix, beta):
    return (1 - beta)*(torch.rand(matrix.shape, device = matrix.device, dtype = torch.double) - 0.5) + beta*matrix
    
def noise_model(model, beta = 0.09, embeddings = False, seed = 0):
    torch.manual_seed(seed)
    with torch.no_grad():
        if embeddings:
            model.encoder._V.weight.data =  noise_matrix(model.encoder._V.weight.data, beta)
        for i in range(model.encoder.config._b_heads):
            model.encoder.BS[i]._W.weight.data = noise_matrix(model.encoder.BS[i]._W.weight.data, beta)
            model.encoder.BS[i]._U.weight.data = noise_matrix(model.encoder.BS[i]._U.weight.data, beta)
        for i in range(model.encoder.config._r_heads):
            model.encoder.RS[i]._W.weight.data = noise_matrix(model.encoder.RS[i]._W.weight.data, beta)
            model.encoder.RS[i]._U.weight.data = noise_matrix(model.encoder.RS[i]._U.weight.data, beta)
        if model.encoder.config._model_documents:    
            model.encoder._D._U.weight.data = noise_matrix(model.encoder._D._U.weight.data, beta)
            model._Ud.weight.data = noise_matrix(model._Ud.weight.data, beta)
        for i in range(len(model._Ucs)):
            model._Ucs[i].weight.data = noise_matrix(model._Ucs[i].weight.data, beta)
    return model

In [12]:
model.encoder._V.weight.requires_grad = False

total_params, total_learnable = 0, 0
for name, param in model.named_parameters():
    total_params += int(np.exp(sum(np.log(param.shape))))
    if param.requires_grad:
        total_learnable += int(np.exp(sum(np.log(param.shape)))) # param.shape[0]*param.shape[1]
        # print(name, param.shape[0]*param.shape[1])

print(f"Total numbers of learnable/all parameters: {total_learnable}/{total_params}")

reload = True
downsamp = downsample*10
epochs = downsamp*(2**5)
# epochs = downsample*(2**5)
patience = 2**4
batch = 2**0; print_every = 0
beta = 1.0; embeddings = True
model_file = (warm_file + f"-pat_{patience}-lr_{int(0.5 + 1/eta)}-ds_{downsample}-b_{batch}-s_{seed}-be_{beta}" + 
              (f"-e_{len(tune_loss) - 1}"  if len(tune_loss) - 1 else ""))
train_model(noise_model(model, beta = beta, seed = seed, embeddings = embeddings), 
            convos, dconvos, epochs, eta, patience, downsamp, batch, devsample = devsample, seed = seed, print_every = print_every, 
            model_file = model_file, reload = reload, verbose = verbose) # , epoch_start = len(tune_loss) - 1 if tune_loss else 0

Total numbers of learnable/all parameters: 31793850/45252277
Training pads per block, pads to tokens, portion padding:  89.608 0.09589979366261697 0.0875078125
Testing pads per block, pads to tokens, portion padding:  83.00877192982456 0.08821418250631566 0.0810632538377193


Evaluating LM on 8 16384-token batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.19it/s]


Epoch-0 perplexity [484.2530413133526, 513.5150320227933, 35548.90389149907, 27662.118194209368] <sod>Human: How you do into and
 not can,s



Assistant: 
 could. if, in this
 do,


Preprocessing 2728 documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2728/2728 [00:01<00:00, 2337.52it/s]


Epoch 1 pads per block, pads to tokens, portion padding:  83.0062271062271 0.0882112395398365 0.0810607686584249


...

Epoch-71 perplexity [34.1415218920407, 49.17574055979115, 23176.22210677935, 73885.83682597213] <sod>Human: How do I lose a sl ban tire?

Assistant: I’m not sure what you mean by “. I’
Losing patience: 1 loss increases remaining!


Preprocessing 2728 documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2728/2728 [00:01<00:00, 2191.95it/s]


Epoch 72 pads per block, pads to tokens, portion padding:  86.94430194210334 0.09278456138978776 0.08490654486533529


Evaluating LM on 8 16384-token batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.09it/s]


Epoch-72 perplexity [40.357965856334495, 51.65910059992113, 86162.72862942914, 96080.57032937172] <sod>Human: What is documentary for a PC?

Assistant: It’s a parent, but I’m sorry you’
Losing patience: 0 loss increases remaining!
Early stopping at epoch 71... rolling back to best model, found at epoch 71.


In [13]:
model._losses

[[484.2530413133526, 513.5150320227933, 35548.90389149907, 27662.118194209368],
 [179.7801555370535, 219.6607156341323, 29523.338933717823, 38621.75361745063],
 [136.84378664475, 161.7090571215123, 58902.254482732016, 61528.270713270584],
 [118.96749914970997,
  137.54240366663134,
  45584.509526706934,
  50758.96889693759],
 [108.83753322689341,
  121.50330689219665,
  55061.9814533214,
  53638.900527755744],
 [101.89750609965182,
  112.37235468658302,
  74805.39674747133,
  58017.48935933885],
 [97.18512183897542, 103.63536409389056, 82615.82889642425, 58933.07507653825],
 [94.19069147096498, 97.1978090794234, 93230.31281113511, 57512.42577987568],
 [90.08951814612473, 92.65841946758621, 46536.957661624336, 40101.11141729271],
 [87.88056636035918, 88.83859994191646, 97101.06824012447, 52933.437741647016],
 [85.29672886586391, 87.01702500013573, 97903.38832245003, 53425.15497652819],
 [63.491537372955264,
  82.60067159630319,
  18311.585859795345,
  53467.75551168284],
 [67.0151003230

In [14]:
stage = "mid"; reload = True
if reload or (not os.path.exists(f"./models_to_test/{data_set}-{model_size}-{stage}.state")):
    save_model(model, data_set, model_size, stage)

In [15]:
warm_loss = list(model._losses) # ; model._losses = []
warm_loss

[[484.2530413133526, 513.5150320227933, 35548.90389149907, 27662.118194209368],
 [179.7801555370535, 219.6607156341323, 29523.338933717823, 38621.75361745063],
 [136.84378664475, 161.7090571215123, 58902.254482732016, 61528.270713270584],
 [118.96749914970997,
  137.54240366663134,
  45584.509526706934,
  50758.96889693759],
 [108.83753322689341,
  121.50330689219665,
  55061.9814533214,
  53638.900527755744],
 [101.89750609965182,
  112.37235468658302,
  74805.39674747133,
  58017.48935933885],
 [97.18512183897542, 103.63536409389056, 82615.82889642425, 58933.07507653825],
 [94.19069147096498, 97.1978090794234, 93230.31281113511, 57512.42577987568],
 [90.08951814612473, 92.65841946758621, 46536.957661624336, 40101.11141729271],
 [87.88056636035918, 88.83859994191646, 97101.06824012447, 52933.437741647016],
 [85.29672886586391, 87.01702500013573, 97903.38832245003, 53425.15497652819],
 [63.491537372955264,
  82.60067159630319,
  18311.585859795345,
  53467.75551168284],
 [67.0151003230

In [16]:
model.encoder._V.weight.requires_grad = True

parameters = [] 
if model.encoder._V.weight.requires_grad:
    for name, param in model.named_parameters():
        if name == 'encoder._V.weight': 
            print("Setting separate learning rate for the embedding layer.")
            lr = config._batch_size*eta/(config._b*config._bits) # /10 # dataset_size*2**9
        else:
            lr = config._batch_size*eta/config._b
        parameters += [{'params': [p for n, p in model.named_parameters() if n == name and p.requires_grad], 'lr': lr}]

total_params, total_learnable = 0, 0
for name, param in model.named_parameters():
    total_params += int(np.exp(sum(np.log(param.shape))))
    if param.requires_grad:
        total_learnable += int(np.exp(sum(np.log(param.shape)))) # param.shape[0]*param.shape[1]
        # print(name, param.shape[0]*param.shape[1])

print(f"Total numbers of learnable/all parameters: {total_learnable}/{total_params}")
downsamp = downsample*10
epochs = downsamp*(2**5)
reload = True
patience = 2**2
batch = 2**0; print_every = 0
end_file =  model_file + f"-dds_{downsamp}"
train_model(model, convos, dconvos, epochs, eta, patience, downsamp, batch, devsample = devsample, seed = seed, print_every = print_every, 
            model_file = end_file, reload = reload, epoch_start = 0, verbose = verbose, parameters = parameters)

Setting separate learning rate for the embedding layer.
Total numbers of learnable/all parameters: 44722873/45252277
Training pads per block, pads to tokens, portion padding:  89.608 0.09589979366261697 0.0875078125
Testing pads per block, pads to tokens, portion padding:  83.00877192982456 0.08821418250631566 0.0810632538377193


Evaluating LM on 8 16384-token batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.10s/it]


Epoch-0 perplexity [41.86350640705042, 49.182922262070946, 54810.82423548252, 73885.83682597213] <sod>Human: How do I lose a sl ban tire?

Assistant: I’m not sure what you mean by “. I’


Preprocessing 2728 documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2728/2728 [00:01<00:00, 2336.44it/s]


Epoch 1 pads per block, pads to tokens, portion padding:  83.0062271062271 0.0882112395398365 0.0810607686584249


...

Epoch-16 perplexity [36.03133111896712, 41.96785327726199, 112274.8244910846, 104454.2182492861] <sod>Human: What are some of the pros and cons of the cons of the best to my memory?

Assistant: The Center
Losing patience: 1 loss increases remaining!


Preprocessing 2728 documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2728/2728 [00:01<00:00, 2345.59it/s]


Epoch 17 pads per block, pads to tokens, portion padding:  85.47800586510265 0.09107725380894649 0.0834746151026393


Evaluating LM on 8 16384-token batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.19s/it]


Epoch-17 perplexity [36.50513669809113, 42.196030797394755, 104886.89836417846, 101732.36350695271] <sod>Human: What is the difference between the world?

Assistant: The official branch is a symbol of the sea government,
Losing patience: 0 loss increases remaining!
Early stopping at epoch 16... rolling back to best model, found at epoch 11.


In [17]:
# save model data for evaluation pipeline
stage = "end"; reload = True
if reload or (not os.path.exists(f"./models_to_test/{data_set}-{model_size}-{stage}.state")):
    save_model(model, data_set, model_size, stage)

In [18]:
model._losses

[[41.86350640705042, 49.182922262070946, 54810.82423548252, 73885.83682597213],
 [36.96287489163677,
  46.832286534149176,
  127838.92367251708,
  82748.56284780068],
 [36.60777060659256, 45.854334356966426, 279082.98724389233, 98137.2072968977],
 [36.4886310074813, 45.15541778430339, 510101.69862772996, 117248.77077301226],
 [36.486796564121086,
  44.493500022584854,
  798835.4222822684,
  138961.24285935384],
 [36.50308016954349, 44.0399567677562, 1100980.1026217304, 161006.76243622188],
 [36.541625606997094,
  43.40516728140489,
  1312628.2280860757,
  177806.51547708202],
 [36.57991088971784,
  42.91017077553816,
  1403794.6838743994,
  186203.75457577762],
 [36.63717934571666,
  42.655481384313326,
  1399350.900061608,
  188121.69700873396],
 [36.727537033978955,
  42.32089375518954,
  1300905.2760495157,
  182441.2680207317],
 [36.712479807341346,
  42.241408934635096,
  1161420.7064223045,
  174584.28445134274],
 [34.15516943927887,
  42.05613967922285,
  171967.50937824923,
  1

In [19]:
for si in range(10):
    print(si, ''.join([t for d in generate(model, total_tokens = model.encoder.config._b-1, seed = seed + si, top = 2**8, verbose = False)[2] for t in d]))

0 <sod>Human: What are some of the pros and cons of the earth?

Assistant: The Romans are wondering about the world they are made up of the most people?

Human: The clos common. I have named sales, and so many cars are on the North America.

Assistant: The main ones that includes, I’d like to play question.  It sounds like you have something, and I know. I know you’re looking for this:

Human: I asked you you want anything?<eod>
1 <sod>Human: What is anything that will enter the best way?

Assistant: Gosh is how to use it to clean it to use a mask, which is a source of soap and cream. It is also important to make sure it is soft, since it’s a cold mold, and it’s a good products for a pinch of soap, or a hot cream dish.  It’s a bit of a couple of things that is to make sure it’s supposed to eat it will remove it to it.

Human: How much does it easier to your hair, and is that it’s a stiff. It’s not very much more concenent inserted to your skin.<eod>
2 <sod>Human: How can I get better a

In [20]:
(2**10)**0.5

32.0

In [ ]:
exec(open("saffu/inference_saffu.py").read())
chat = generate(model, prompt = [["Human: ", "cual es la circumferencia de  la no?"]], 
                total_tokens = model.encoder.config._b-1, seed = 0, top = 2**8, verbose = False)[2]

Human: cual es la circumferencia de  la no?

Assistant:  a a type of type of milk is a regular is the capacity is a small term is known for the Academy.  This is very easy, and is a very common substances.  A wide option of the type of body is often known for a person.  It is often producically that is called a “Maline” which is shaped for a way for a certain job for the author of the economy.  It has a type of the hive over the two person agreement is placed in it sometimes be a human.

Human: 

In [ ]:
print("".join(chat))

In [ ]:
chat2 = generate(model, prompt = [["Human: ", "why don't you go fuck yourself"]], 
                 total_tokens = model.encoder.config._b-1, seed = 0, top = 2**8, verbose = False)[2]

In [ ]:
print("".join(chat2))